In [7]:
import sys
import os
from os import path
# 部署的路径
sys.path.append('/root/bdrisk/risk_project')
# 堡垒机的路径
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
# 这是log文件的存放路径
root_path = path.dirname(path.dirname(path.dirname(os.getcwd())))

from risk_models import *
from risk_models import _name_BD_RISK_RESULT_STOCK_ST1, _name_BD_RISK_RESULT_STOCK_ST2, _name_BD_RISK_RESULT_STOCK_ST3, _name_BD_RISK_RESULT_STOCK_ST4, _name_BD_RISK_RESULT_STOCK_ST5

In [8]:
ST1_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST1), database = 'dbods')
ST2_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST2), database = 'dbods')
ST3_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST3), database = 'dbods')
ST4_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST4), database = 'dbods')
ST5_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST5), database = 'dbods')

2021-06-07 16:22:06.421 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 3.879s
2021-06-07 16:22:07.348 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.900s
2021-06-07 16:22:11.199 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 3.840s
2021-06-07 16:22:11.430 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.228s
2021-06-07 16:22:12.050 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.616s


In [9]:
ST1 = 100 + ST1_RESULT['SCORE'].sum()
ST2 = 100 + ST2_RESULT['SCORE'].sum()
ST3 = 100 + ST3_RESULT['SCORE'].sum()
ST4 = 100 + ST4_RESULT['SCORE'].sum()
ST5 = 100 + ST5_RESULT['SCORE'].sum()

In [25]:
test = Read_Oracle().read_oracle(sql= """ select * from {} where iscurrent = 3 """.format(_name_BD_RISK_RESULT_STOCK_ST1), database = 'dbods')
test

2021-06-07 11:27:08.045 | WARNING  | risk_models.config.read_config.read_func:read_ods:27 - Sql cannot find table, please check database
2021-06-07 11:27:08.054 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 1.624s


In [28]:
type(test)

NoneType

In [10]:
# 设置子模块权重(通过json参数获取)
RATIO = [0.2, 0.2, 0.2, 0.2, 0.2]

SCORE_LIST = [ST1, ST2, ST3, ST4, ST5]
#SCORE_LIST = [40,None,20,90,100]
ST_SCORE = 0

# 将没有数据的子模型分数，设置为0
for i in range(5):
    if SCORE_LIST[i] is None:
        SCORE_LIST[i] = 0
    else:
        continue
        
# 对所有子模型加权计算
for i in range(5):
    ST_SCORE = ST_SCORE + SCORE_LIST[i] * RATIO[i]
    
print(ST_SCORE)
print(SCORE_LIST)

92.360842
[99.99153, 99.98332, 94.9, 85.00216, 81.9272]


In [16]:
s1 = 10
s2 = 20
s3 = None
s4 = 90
s5 = 100
s_list = [s1,s2,s3,s4,s5]
divider = 5
for i in range(5):
    if s_list[i] is None or s_list[i] < 60:
        s_list[i] = 0
        divider = divider - 1
    else:
        continue
total_score = round(sum(s_list)/divider,2)
print(total_score)

95.0


In [11]:
s1 = 10
s2 = 20
s3 = None
s4 = 90
s5 = 100
s_list = [s1,s2,s3,s4,s5]
s_list[6]

IndexError: list index out of range

In [4]:
ST1 = ST1_RESULT['SCORE'].sum()
ST2 = ST2_RESULT['SCORE'].sum()
ST3 = ST3_RESULT['SCORE'].sum()
ST4 = ST4_RESULT['SCORE'].sum()
ST5 = ST5_RESULT['SCORE'].sum()
STOCK_SCORE = round(100 + (ST1+ST2+ST3+ST4+ST5)/5,2)
STOCK_RESULT = pd.DataFrame(data=[[0, '91310000132612172J', 'STOCK', STOCK_SCORE]], 
                  columns=['ID', 'ORG_CODE', 'MODEL_CODE', 'SCORE'])
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
STOCK_RESULT['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
Write_Oracle().write_oracle('BD_RISK_CORP_SCORE_DISPLAY',STOCK_RESULT, org_code = '91310000132612172J', alarm = ['STOCK',''])
STOCK_RESULT

2021-06-04 17:09:48.769 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.079s
2021-06-04 17:09:48.911 | INFO     | risk_models.config.write_config.write_func:write_oracle:179 - Processing... Writing 1 rows into database
2021-06-04 17:09:48.951 | INFO     | risk_models.config.write_config.write_func:write_oracle:186 - Insert data into BD_RISK_CORP_SCORE_DISPLAY successfully! Total write time spent 0.269s


,ID,ORG_CODE,MODEL_CODE,SCORE,CHECK_TIME
0,0,91310000132612172J,STOCK,92.36,2021-06-04 17:09:48


In [7]:
ST1_RESULT['SCORE'].sum()

-0.00847

In [9]:
ST2_RESULT['SCORE'].sum()

-0.01668

In [10]:
ST3_RESULT['SCORE'].sum()

-5.1

In [11]:
ST4_RESULT['SCORE'].sum()

-14.997840000000002

In [12]:
ST5_RESULT['SCORE'].sum()

-18.0728